In [1]:
# Code to work in Google Colab
%load_ext autoreload
%autoreload 2

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Leave this
num_epochs = 10

# TUNE THESE to achieve maximum performance
batch_size = 16
learning_rate = 1e-3

In [3]:
import os

GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', 'EECS 545 Project')
print(os.listdir(GOOGLE_DRIVE_PATH))

os.chdir(GOOGLE_DRIVE_PATH)
print(os.getcwd())

['metadata.json', 'Image.zip', '545 Project Ideas.gdoc', 'data_scene_flow', 'Copy of Data_Loading.ipynb', '.ipynb_checkpoints', 'Image.zip (Unzipped Files)', 'Images', 'Untitled Diagram.drawio', 'Weather', 'Extra_Weather_Data', 'More_Weather_Data_60k', 'DataSplit.ipynb', 'Unet + triplet loss.ipynb', 'Data_Loading.ipynb', 'saved_model', 'Untitled', '__pycache__', 'Test version(Yuanbin)-UNet Triplet loss.ipynb', 'JoeyVAE598.py', 'Performance', 'WeatherVAE.ipynb', 'Joey Unet and Content Loss.ipynb', 'WeatherVAE.pt', 'ExampleInterpolation.png', 'Tune parameters.ipynb', 'WeatherVAE-test (Jason).ipynb', 'newest Unet and Triplet Loss.ipynb', 'JoeyTransferLearning.ipynb', 'Baselines.ipynb']
/content/drive/My Drive/EECS 545 Project


In [4]:
import pandas as pd
import torch.nn as nn
from PIL import Image
from torchvision import transforms

import torch
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [5]:
class SmallDataLoader:
  def __init__(self, BatchSize=16, TestPercent=0.1, data_loc="Weather", img_size=224, dev="cuda"):
    self.train_names = []
    self.train_labels = []
    self.val_names = []
    self.val_labels = []
    self.test_names = []
    self.test_labels = []

    self.label_keys = {0: "cloudy", 1: "foggy", 2: "rain", 3: "snow", 4: "sunny"}
    self.data_loc = os.path.join(os.getcwd(), data_loc)
    self.batch_size = BatchSize
    self.img_size = img_size
    self.dev=dev

    data_types = ["cloudy", "foggy", "rain", "snow", "sunny"]
    splits = ["Train", "Val", "Test"]
    for split in splits:
      for i in range(5):
        dtype = self.label_keys[i]
        type_loc = os.path.join(data_loc, split, dtype)
        files = os.listdir(type_loc)

        if split == "Train":
          self.train_labels += len(files) * [i]
          self.train_names += files
        elif split == "Val":
          self.val_labels += len(files) * [i]
          self.val_names += files
        elif split == "Test":
          self.test_labels += len(files) * [i]
          self.test_names += files

    # Shuffle data first
    self.N_train = len(self.train_names) - len(self.train_names) % self.batch_size
    self.N_val = len(self.val_names) - len(self.val_names) % self.batch_size
    self.N_test = len(self.test_names) - len(self.test_names) % self.batch_size
    self.shuffle_train()

    self.iter_no = 0
    self.val_no = 0

    self.transform_dict = {
        "Train": transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),
        transforms.GaussianBlur(5),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]),
        "Val": transforms.Compose([
        transforms.CenterCrop(img_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
    }

  # index into file names, labels
  def get_ind(self, index, dset="Train"):
    if dset == "Train":
      fname = self.train_names[index]
      label = self.train_labels[index]
    elif dset == "Val":
      fname = self.val_names[index]
      label = self.val_labels[index]

    label_name = self.label_keys[label]
    img_loc = os.path.join(self.data_loc, dset, label_name, fname)
    input_image = Image.open(img_loc).convert('RGB')
    input_tensor = self.transform_dict[dset](input_image)

    return input_tensor, label

  # For training
  def get_batch(self):
    # End of Epoch
    if self.iter_no + self.batch_size >= self.N_train:
      self.iter_no = 0
      return "EOE", None

    batch_img = torch.zeros((self.batch_size, 3, self.img_size, self.img_size), device=self.dev)
    batch_lab = torch.zeros((self.batch_size), device=self.dev, dtype=torch.long)
    for i in range(self.batch_size):
      input_tensor, label = self.get_ind(self.iter_no + i)
      batch_img[i] = input_tensor
      batch_lab[i] = label
    self.iter_no += self.batch_size
    return batch_img, batch_lab

  def get_val(self):
    if self.val_no + self.batch_size >= self.N_val:
      self.val_no = 0
      return "EOE", None

    batch_img = torch.zeros((self.batch_size, 3, self.img_size, self.img_size), device=self.dev)
    batch_lab = torch.zeros((self.batch_size), device=self.dev, dtype=torch.long)
    for i in range(self.batch_size):
      input_tensor, label = self.get_ind(self.val_no + i, dset="Val")
      batch_img[i] = input_tensor
      batch_lab[i] = label
    self.val_no += self.batch_size
    return batch_img, batch_lab


  def shuffle_train(self):
    inds = np.random.choice(self.N_train, size=self.N_train, replace=False)
    self.train_names = [self.train_names[i] for i in inds]
    self.train_labels = [self.train_labels[i] for i in inds]
    self.iter_no = 0

In [6]:
class BigDataLoader:
  def __init__(self, BatchSize=16, TrainRat=0.9, ValRat=0.05, TestRat=0.05, file_loc="Weather", data_loc="Images/Image", img_size=224, dev="cuda"):
    self.train_names = []
    self.train_labels = []
    self.val_names = []
    self.val_labels = []
    self.test_names = []
    self.test_labels = []

    self.label_keys = {0: "cloudy", 1: "foggy", 2: "rain", 3: "snow", 4: "sunny"}
    self.data_loc = os.path.join(os.getcwd(), data_loc)
    self.file_loc = os.path.join(os.getcwd(), file_loc)
    self.batch_size = BatchSize
    self.img_size = img_size
    self.dev=dev

    # Get file names for weather types
    for w_ind in self.label_keys.keys():
      w_type = self.label_keys[w_ind]
      f = open(os.path.join(file_loc, w_type + ".txt"), 'r')
      lines = [line.strip() for line in f.readlines()]
      f.close()
      # Create a list of keys, files
      N = len(lines)
      self.train_names += lines[:int(N*TrainRat)]
      self.train_labels += [w_ind] * len(lines[:int(N*TrainRat)])

      self.val_names += lines[int(N*TrainRat):int(N*(TrainRat + ValRat))]
      self.val_labels += [w_ind] * len(lines[int(N*TrainRat):int(N*(TrainRat + ValRat))])

      self.test_names += lines[int(N*(TrainRat + ValRat)):]
      self.test_labels += [w_ind] * len(lines[int(N*(TrainRat + ValRat)):])

    # Shuffle data first
    self.N_train = len(self.train_names) - len(self.train_names) % self.batch_size
    self.N_val = len(self.val_names) - len(self.val_names) % self.batch_size
    self.N_test = len(self.test_names) - len(self.test_names) % self.batch_size
    self.shuffle_train()

    self.iter_no = 0
    self.val_no = 0

    self.transform_dict = {
        "Train": transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),
        transforms.GaussianBlur(5),
         transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]),
        "Val": transforms.Compose([
        transforms.CenterCrop(img_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
    }

  # index into file names, labels
  def get_ind(self, index, dset="Train"):
    if dset == "Train":
      fname = self.train_names[index]
      label = self.train_labels[index]
    elif dset == "Val":
      fname = self.val_names[index]
      label = self.val_labels[index]

    label_name = self.label_keys[label]
    img_loc = os.path.join(self.data_loc, label_name, fname)
    input_image = Image.open(img_loc)
    input_tensor = self.transform_dict[dset](input_image)

    return input_tensor, label

  # For training
  def get_batch(self):
    # End of Epoch
    if self.iter_no + self.batch_size >= self.N_train:
      self.iter_no = 0
      return "EOE", None

    batch_img = torch.zeros((self.batch_size, 3, self.img_size, self.img_size), device=self.dev)
    batch_lab = torch.zeros((self.batch_size), device=self.dev, dtype=torch.long)
    for i in range(self.batch_size):
      input_tensor, label = self.get_ind(self.iter_no + i)
      batch_img[i] = input_tensor
      batch_lab[i] = label
    self.iter_no += self.batch_size
    return batch_img, batch_lab

  def get_val(self):
    if self.val_no + self.batch_size >= self.N_val:
      self.val_no = 0
      return "EOE", None

    batch_img = torch.zeros((self.batch_size, 3, self.img_size, self.img_size), device=self.dev)
    batch_lab = torch.zeros((self.batch_size), device=self.dev, dtype=torch.long)
    for i in range(self.batch_size):
      input_tensor, label = self.get_ind(self.val_no + i, dset="Val")
      batch_img[i] = input_tensor
      batch_lab[i] = label
    self.val_no += self.batch_size
    return batch_img, batch_lab


  def shuffle_train(self):
    inds = np.random.choice(self.N_train, size=self.N_train, replace=False)
    self.train_names = [self.train_names[i] for i in inds]
    self.train_labels = [self.train_labels[i] for i in inds]
    self.iter_no = 0

In [7]:
dl = SmallDataLoader()
dl.shuffle_train()
print(dl.N_train, dl.N_val, dl.N_test)

# dl = BigDataLoader()
# dl.shuffle_train()
# print(dl.N_train, dl.N_val, dl.N_test)

2144 256 384


In [8]:
# Example
model = torch.hub.load('pytorch/vision:v0.9.0', 'resnet18', pretrained=True)
num_ftrs = model.fc.in_features
num_classes = len(dl.label_keys)
model.fc = nn.Linear(num_ftrs, num_classes)
model.to(dl.dev)

batch_im, batch_lab = dl.get_batch()
with torch.no_grad():
    output = model(batch_im)

# Tensor of shape 5
print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0


tensor([0.8283, 1.2342, 0.2331, 0.2383, 0.1288], device='cuda:0')


In [9]:
def train_model(model, criterion, optimizer, scheduler, data_loader, num_epochs=10):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    train_hist = []
    val_hist = []

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            data_loader.shuffle_train()
            while True:
                if phase == "train":
                  inputs, labels = data_loader.get_batch()
                else:
                  inputs, labels = data_loader.get_val()
                # End of epoch
                if inputs == "EOE":
                  break

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()
            if phase == "train":
              epoch_loss = running_loss / data_loader.N_train
              epoch_acc = running_corrects / data_loader.N_train
            else:
              epoch_loss = running_loss / data_loader.N_val
              epoch_acc = running_corrects / data_loader.N_val

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            if phase == "train":
              train_hist.append(epoch_acc)
            else:
              val_hist.append(epoch_acc)

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, train_hist, val_hist

In [10]:
# Without transfer learning
dl = SmallDataLoader(BatchSize=batch_size)

models = {}
train_hist = {}
val_hist = {}
num_classes = len(dl.label_keys)

for model_name in ["resnet50", "vgg11_bn", "vgg19_bn", "resnet18", "resnet34"]:
  print(model_name)
  model = torch.hub.load('pytorch/vision:v0.9.0', model_name, pretrained=False)
  if model_name[:3] == "vgg":
    num_ftrs = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(num_ftrs, num_classes)
  else:
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)
  model.to(dl.dev)

  criterion = nn.CrossEntropyLoss()

  # Observe that all parameters are being optimized
  optimizer_ft = optim.Adam(model.parameters(), lr=learning_rate)

  # Decay LR by a factor of 0.1 every 7 epochs
  exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=1, gamma=0.1)

  models[model_name], train_hist[model_name], val_hist[model_name] = train_model(model, criterion, optimizer_ft, exp_lr_scheduler, dl, num_epochs=num_epochs)

resnet50


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0


Epoch 0/9
----------
train Loss: 1.6768 Acc: 0.3568
val Loss: 1.4199 Acc: 0.3594
Epoch 1/9
----------
train Loss: 1.3411 Acc: 0.4398
val Loss: 1.3380 Acc: 0.3828
Epoch 2/9
----------
train Loss: 1.2719 Acc: 0.4729
val Loss: 1.3363 Acc: 0.3555
Epoch 3/9
----------
train Loss: 1.2827 Acc: 0.4757
val Loss: 1.3313 Acc: 0.3594
Epoch 4/9
----------
train Loss: 1.2864 Acc: 0.4734
val Loss: 1.3346 Acc: 0.3555
Epoch 5/9
----------
train Loss: 1.2773 Acc: 0.4851
val Loss: 1.3374 Acc: 0.3633
Epoch 6/9
----------
train Loss: 1.2888 Acc: 0.4757
val Loss: 1.3330 Acc: 0.3555
Epoch 7/9
----------
train Loss: 1.2687 Acc: 0.4972
val Loss: 1.3349 Acc: 0.3594
Epoch 8/9
----------
train Loss: 1.2878 Acc: 0.4669
val Loss: 1.3351 Acc: 0.3516
Epoch 9/9
----------
train Loss: 1.2753 Acc: 0.4711
val Loss: 1.3309 Acc: 0.3555
Training complete in 10m 50s
Best val Acc: 0.382812
vgg11_bn


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0


Epoch 0/9
----------
train Loss: 4.3829 Acc: 0.2341
val Loss: 1.5580 Acc: 0.2344
Epoch 1/9
----------
train Loss: 1.5608 Acc: 0.2519
val Loss: 1.5019 Acc: 0.2930
Epoch 2/9
----------
train Loss: 1.5445 Acc: 0.2407
val Loss: 1.4981 Acc: 0.2773
Epoch 3/9
----------
train Loss: 1.5463 Acc: 0.2645
val Loss: 1.4953 Acc: 0.2812
Epoch 4/9
----------
train Loss: 1.5456 Acc: 0.2668
val Loss: 1.4931 Acc: 0.2695
Epoch 5/9
----------
train Loss: 1.5431 Acc: 0.2696
val Loss: 1.5001 Acc: 0.2773
Epoch 6/9
----------
train Loss: 1.5436 Acc: 0.2691
val Loss: 1.5006 Acc: 0.2773
Epoch 7/9
----------
train Loss: 1.5405 Acc: 0.2663
val Loss: 1.5009 Acc: 0.2695
Epoch 8/9
----------
train Loss: 1.5440 Acc: 0.2607
val Loss: 1.4975 Acc: 0.2969
Epoch 9/9
----------
train Loss: 1.5442 Acc: 0.2561
val Loss: 1.5034 Acc: 0.2891
Training complete in 10m 50s
Best val Acc: 0.296875
vgg19_bn


...

Epoch 0/9
----------
train Loss: 5.5503 Acc: 0.2332
val Loss: 1.6238 Acc: 0.1484
Epoch 1/9
----------
train Loss: 1.6125 Acc: 0.2575
val Loss: 1.5157 Acc: 0.2578
Epoch 2/9
----------
train Loss: 1.5479 Acc: 0.2542
val Loss: 1.5104 Acc: 0.2930
Epoch 3/9
----------
train Loss: 1.5431 Acc: 0.2868
val Loss: 1.5142 Acc: 0.2773
Epoch 4/9
----------
train Loss: 1.5461 Acc: 0.2547
val Loss: 1.5205 Acc: 0.2383
Epoch 5/9
----------
train Loss: 1.5394 Acc: 0.2943
val Loss: 1.5165 Acc: 0.2500
Epoch 6/9
----------
train Loss: 1.5373 Acc: 0.2771
val Loss: 1.5183 Acc: 0.2383
Epoch 7/9
----------
train Loss: 1.5292 Acc: 0.2607
val Loss: 1.5113 Acc: 0.2891
Epoch 8/9
----------
train Loss: 1.5409 Acc: 0.2733
val Loss: 1.5121 Acc: 0.2852
Epoch 9/9
----------
train Loss: 1.5362 Acc: 0.2817
val Loss: 1.5153 Acc: 0.3359
Training complete in 14m 49s
Best val Acc: 0.335938
resnet18


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0


Epoch 0/9
----------
train Loss: 1.5202 Acc: 0.3652
val Loss: 1.3276 Acc: 0.3750
Epoch 1/9
----------
train Loss: 1.3339 Acc: 0.4454
val Loss: 1.3601 Acc: 0.3281
Epoch 2/9
----------
train Loss: 1.2737 Acc: 0.4627
val Loss: 1.3391 Acc: 0.3359
Epoch 3/9
----------
train Loss: 1.2793 Acc: 0.4543
val Loss: 1.3401 Acc: 0.3438
Epoch 4/9
----------
train Loss: 1.2862 Acc: 0.4669
val Loss: 1.3175 Acc: 0.3516
Epoch 5/9
----------
train Loss: 1.2730 Acc: 0.4650
val Loss: 1.3233 Acc: 0.3516
Epoch 6/9
----------
train Loss: 1.2838 Acc: 0.4632
val Loss: 1.3405 Acc: 0.3438
Epoch 7/9
----------
train Loss: 1.2618 Acc: 0.4729
val Loss: 1.3236 Acc: 0.3438
Epoch 8/9
----------
train Loss: 1.2764 Acc: 0.4608
val Loss: 1.3638 Acc: 0.3320
Epoch 9/9
----------
train Loss: 1.2465 Acc: 0.4683
val Loss: 1.3277 Acc: 0.3477
Training complete in 8m 23s
Best val Acc: 0.375000
resnet34


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0


Epoch 0/9
----------
train Loss: 1.5823 Acc: 0.3316
val Loss: 1.5783 Acc: 0.3516
Epoch 1/9
----------
train Loss: 1.3936 Acc: 0.4193
val Loss: 1.3263 Acc: 0.3750
Epoch 2/9
----------
train Loss: 1.3535 Acc: 0.4408
val Loss: 1.3289 Acc: 0.3789
Epoch 3/9
----------
train Loss: 1.3471 Acc: 0.4375
val Loss: 1.3181 Acc: 0.3750
Epoch 4/9
----------
train Loss: 1.3487 Acc: 0.4338
val Loss: 1.3249 Acc: 0.3633
Epoch 5/9
----------
train Loss: 1.3496 Acc: 0.4398
val Loss: 1.3309 Acc: 0.3672
Epoch 6/9
----------
train Loss: 1.3527 Acc: 0.4310
val Loss: 1.3316 Acc: 0.3711
Epoch 7/9
----------
train Loss: 1.3464 Acc: 0.4370
val Loss: 1.3307 Acc: 0.3594
Epoch 8/9
----------
train Loss: 1.3546 Acc: 0.4422
val Loss: 1.3212 Acc: 0.3672
Epoch 9/9
----------
train Loss: 1.3468 Acc: 0.4333
val Loss: 1.3203 Acc: 0.3633
Training complete in 8m 42s
Best val Acc: 0.378906


In [11]:
print("Not transfer learning")
for model_name in train_hist.keys():
  print(model_name, val_hist[model_name][-1])

Not transfer learning
resnet50 tensor(0.3555, device='cuda:0')
vgg11_bn tensor(0.2891, device='cuda:0')
vgg19_bn tensor(0.3359, device='cuda:0')
resnet18 tensor(0.3477, device='cuda:0')
resnet34 tensor(0.3633, device='cuda:0')


In [12]:
# Using transfer learning
dl = SmallDataLoader(BatchSize=batch_size)

models = {}
train_hist = {}
val_hist = {}
num_classes = len(dl.label_keys)

for model_name in ["resnet50", "vgg11_bn", "vgg19_bn", "resnet18", "resnet34"]:
  print(model_name)
  model = torch.hub.load('pytorch/vision:v0.9.0', model_name, pretrained=True)

  # Freeze
  for param in model.parameters():
    param.requires_grad = False

  # Add fc layer
  if model_name[:3] == "vgg":
    num_ftrs = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(num_ftrs, num_classes)
  else:
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)
  model.to(dl.dev)

  criterion = nn.CrossEntropyLoss()

  # Observe that all parameters are being optimized
  optimizer_ft = optim.Adam(model.parameters(), lr=learning_rate)

  # Decay LR by a factor of 0.1 every 7 epochs
  exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=1, gamma=0.1)

  models[model_name], train_hist[model_name], val_hist[model_name] = train_model(model, criterion, optimizer_ft, exp_lr_scheduler, dl, num_epochs=num_epochs)

resnet50


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0


Epoch 0/9
----------
train Loss: 1.1949 Acc: 0.5182
val Loss: 1.2537 Acc: 0.4102
Epoch 1/9
----------
train Loss: 0.9198 Acc: 0.6549
val Loss: 1.1212 Acc: 0.4727
Epoch 2/9
----------
train Loss: 0.8960 Acc: 0.6712
val Loss: 1.1447 Acc: 0.4570
Epoch 3/9
----------
train Loss: 0.8906 Acc: 0.6758
val Loss: 1.1526 Acc: 0.4453
Epoch 4/9
----------
train Loss: 0.8924 Acc: 0.6660
val Loss: 1.1543 Acc: 0.4688
Epoch 5/9
----------
train Loss: 0.8874 Acc: 0.6782
val Loss: 1.1668 Acc: 0.4492
Epoch 6/9
----------
train Loss: 0.8981 Acc: 0.6665
val Loss: 1.1548 Acc: 0.4375
Epoch 7/9
----------
train Loss: 0.8861 Acc: 0.6800
val Loss: 1.1540 Acc: 0.4609
Epoch 8/9
----------
train Loss: 0.8743 Acc: 0.6810
val Loss: 1.1446 Acc: 0.4492
Epoch 9/9
----------
train Loss: 0.8877 Acc: 0.6758
val Loss: 1.1577 Acc: 0.4375
Training complete in 8m 19s
Best val Acc: 0.472656
vgg11_bn


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0


Epoch 0/9
----------
train Loss: 1.0749 Acc: 0.5835
val Loss: 1.2309 Acc: 0.4375
Epoch 1/9
----------
train Loss: 0.9218 Acc: 0.6283
val Loss: 1.1687 Acc: 0.4688
Epoch 2/9
----------
train Loss: 0.8877 Acc: 0.6642
val Loss: 1.1545 Acc: 0.4766
Epoch 3/9
----------
train Loss: 0.8894 Acc: 0.6446
val Loss: 1.1627 Acc: 0.4570
Epoch 4/9
----------
train Loss: 0.8701 Acc: 0.6730
val Loss: 1.1580 Acc: 0.4727
Epoch 5/9
----------
train Loss: 0.8723 Acc: 0.6632
val Loss: 1.1654 Acc: 0.4570
Epoch 6/9
----------
train Loss: 0.8810 Acc: 0.6581
val Loss: 1.1514 Acc: 0.4766
Epoch 7/9
----------
train Loss: 0.8818 Acc: 0.6572
val Loss: 1.1755 Acc: 0.4570
Epoch 8/9
----------
train Loss: 0.8928 Acc: 0.6516
val Loss: 1.1836 Acc: 0.4531
Epoch 9/9
----------
train Loss: 0.8603 Acc: 0.6702
val Loss: 1.1868 Acc: 0.4609
Training complete in 7m 55s
Best val Acc: 0.476562
vgg19_bn


...

Epoch 0/9
----------
train Loss: 1.1199 Acc: 0.5480
val Loss: 1.3095 Acc: 0.4102
Epoch 1/9
----------
train Loss: 0.9379 Acc: 0.6451
val Loss: 1.3399 Acc: 0.3633
Epoch 2/9
----------
train Loss: 0.9237 Acc: 0.6469
val Loss: 1.2797 Acc: 0.4023
Epoch 3/9
----------
train Loss: 0.9403 Acc: 0.6325
val Loss: 1.2914 Acc: 0.4062
Epoch 4/9
----------
train Loss: 0.9207 Acc: 0.6488
val Loss: 1.3484 Acc: 0.3633
Epoch 5/9
----------
train Loss: 0.9044 Acc: 0.6604
val Loss: 1.2911 Acc: 0.3984
Epoch 6/9
----------
train Loss: 0.9047 Acc: 0.6460
val Loss: 1.3810 Acc: 0.3477
Epoch 7/9
----------
train Loss: 0.9342 Acc: 0.6287
val Loss: 1.3029 Acc: 0.3945
Epoch 8/9
----------
train Loss: 0.9066 Acc: 0.6493
val Loss: 1.3080 Acc: 0.3984
Epoch 9/9
----------
train Loss: 0.9567 Acc: 0.6185
val Loss: 1.3278 Acc: 0.3828
Training complete in 8m 55s
Best val Acc: 0.410156
resnet18


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0


Epoch 0/9
----------
train Loss: 1.3098 Acc: 0.4669
val Loss: 1.3445 Acc: 0.3320
Epoch 1/9
----------
train Loss: 1.0607 Acc: 0.5984
val Loss: 1.3082 Acc: 0.4023
Epoch 2/9
----------
train Loss: 1.0227 Acc: 0.6292
val Loss: 1.3316 Acc: 0.3750
Epoch 3/9
----------
train Loss: 1.0337 Acc: 0.6161
val Loss: 1.3228 Acc: 0.3633
Epoch 4/9
----------
train Loss: 1.0436 Acc: 0.6138
val Loss: 1.3337 Acc: 0.3555
Epoch 5/9
----------
train Loss: 1.0313 Acc: 0.6110
val Loss: 1.3309 Acc: 0.3750
Epoch 6/9
----------
train Loss: 1.0293 Acc: 0.6227
val Loss: 1.3154 Acc: 0.3906
Epoch 7/9
----------
train Loss: 1.0278 Acc: 0.6231
val Loss: 1.3108 Acc: 0.3945
Epoch 8/9
----------
train Loss: 1.0300 Acc: 0.6259
val Loss: 1.3157 Acc: 0.3906
Epoch 9/9
----------
train Loss: 1.0370 Acc: 0.6077
val Loss: 1.3219 Acc: 0.3633
Training complete in 7m 5s
Best val Acc: 0.402344
resnet34


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0


Epoch 0/9
----------
train Loss: 1.2823 Acc: 0.4711
val Loss: 1.3939 Acc: 0.3828
Epoch 1/9
----------
train Loss: 1.0401 Acc: 0.6110
val Loss: 1.3113 Acc: 0.3984
Epoch 2/9
----------
train Loss: 1.0010 Acc: 0.6343
val Loss: 1.2891 Acc: 0.4102
Epoch 3/9
----------
train Loss: 0.9929 Acc: 0.6437
val Loss: 1.3005 Acc: 0.4102
Epoch 4/9
----------
train Loss: 1.0000 Acc: 0.6278
val Loss: 1.2954 Acc: 0.4258
Epoch 5/9
----------
train Loss: 1.0168 Acc: 0.6236
val Loss: 1.2797 Acc: 0.4258
Epoch 6/9
----------
train Loss: 1.0237 Acc: 0.6189
val Loss: 1.2939 Acc: 0.4180
Epoch 7/9
----------
train Loss: 1.0097 Acc: 0.6371
val Loss: 1.2969 Acc: 0.3984
Epoch 8/9
----------
train Loss: 1.0032 Acc: 0.6269
val Loss: 1.2747 Acc: 0.4375
Epoch 9/9
----------
train Loss: 1.0110 Acc: 0.6427
val Loss: 1.2719 Acc: 0.4258
Training complete in 7m 31s
Best val Acc: 0.437500


In [13]:
print("Transfer Learning")
for model_name in train_hist.keys():
  print(model_name, val_hist[model_name][-1])

Transfer Learning
resnet50 tensor(0.4375, device='cuda:0')
vgg11_bn tensor(0.4609, device='cuda:0')
vgg19_bn tensor(0.3828, device='cuda:0')
resnet18 tensor(0.3633, device='cuda:0')
resnet34 tensor(0.4258, device='cuda:0')
